In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import src.model as modelfunc
import src.sequencing as sequencing
from config import config

In [ ]:
print("TensorFlow Version:", tf.__version__)
print("Is TensorFlow built with CUDA?", tf.test.is_built_with_cuda())
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("GPU Details:", tf.config.list_physical_devices('GPU'))


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if tf.config.list_physical_devices('GPU'):
    print("Running on GPU")
else:
    print("GPU not available. Running on CPU")

## parse GPMF

In [ ]:
import telemetry_parser
import pandas as pd

def parse_telemetry(file_path):
    # Convert PosixPath to string if necessary
    tp = telemetry_parser.Parser(str(file_path))
    print(f"Camera: {tp.camera}, Model: {tp.model}")
    
    # Extract and normalize telemetry data
    data = tp.normalized_imu()
    df = pd.DataFrame(data)
    
    # Expand and organize columns
    df[['GYRO_x', 'GYRO_y', 'GYRO_z']] = pd.DataFrame(df['gyro'].tolist(), index=df.index)
    df[['ACCL_x', 'ACCL_y', 'ACCL_z']] = pd.DataFrame(df['accl'].tolist(), index=df.index)
    df.drop(columns=['gyro', 'accl'], inplace=True)
    df = df[['timestamp_ms', 'GYRO_x', 'GYRO_y', 'GYRO_z', 'ACCL_x', 'ACCL_y', 'ACCL_z']]
    
    return df

# Example usage
df = parse_telemetry(config.DATA_DIR / "cosmo_sample.MP4")
display(df)


In [ ]:
df = pd.read_csv(config.DATA_DIR / "test_imu_frameindex_label_data.csv")  # Replace with your data file
overlap = 0.0
length = 10
num_actions = 5  # Example number of actions
df.dropna(inplace=True)

display(df)

# Convert LABEL to categorical and save mappings
df["LABEL"] = df["LABEL"].astype("category")
label_mapping = dict(enumerate(df["LABEL"].cat.categories))
df["LABEL"] = df["LABEL"].cat.codes

display(df)
print(label_mapping)

## Load data

In [ ]:
# Preprocess data
# padded_sequences, padded_labels = sequencing.get_sequences_and_labels(df, overlap, length, num_actions)
sequences = sequencing.create_sequence(df, overlap, length)
padded_sequences = sequencing.get_sequences_pure_data(sequences)
padded_labels = sequencing.get_pure_labels(sequences)
padded_labels = tf.one_hot(padded_labels, depth=5)

In [ ]:
validating = True

if validating:
    train_size = int(0.5 * len(padded_sequences))
    padded_sequences, X_val = padded_sequences[:train_size], padded_sequences[train_size:]
    padded_labels, y_val = padded_labels[:train_size], padded_labels[train_size:]

    print("Train shape:", padded_sequences.shape, padded_labels.shape)
    print("Validation shape:", X_val.shape, y_val.shape)
else:
    X_val = None
    y_val = None

In [11]:
# Generate sample weights to ignore padded timesteps
sample_weights = np.array([
    [1 if np.any(timestep != 0) else 0 for timestep in sequence]
    for sequence in padded_sequences
])

## Build Model

In [ ]:
# Model parameters
print(padded_sequences.shape)
timesteps = padded_sequences.shape[1] # Variable-length sequences
features = 6  # IMU features (e.g., ax, ay, az, gx, gy, gz)
num_classes = 5  # Actions (e.g., left turn, right turn, stopping)
print(timesteps)
# Build the model
model = modelfunc.build_seq2seq_lstm((timesteps, features), num_classes, dropout=0.4)
model.summary()

## Train Model

In [ ]:
# _ = model.predict(X_train[:1])  # Perform a forward pass to initialize the model

# # Build the model explicitly
# model.build(input_shape=(None, timesteps, 6))  # Example: batch size = None, 50 timesteps, 6 features
# model.make_train_function()
# model.make_predict_function()

In [ ]:
# # Example call to the function
# history = modelfunc.train_model(
#     model=model,  # Your pre-defined Keras model
#     X_train=padded_sequences, 
#     y_train=padded_labels,
#     sample_weight=sample_weights,  # Optional; pass None if not using sample weights
#     batch_size=16,  # Optional; defaults to 16
#     epochs=50       # Optional; defaults to 10
# )

# Example call to the function
history = modelfunc.train_model(
    model=model,  # Your pre-defined Keras model
    X_train=X_train, 
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    sample_weight=sample_weights,  # Optional; pass None if not using sample weights
    batch_size=16,  # Optional; defaults to 16
    epochs=10       # Optional; defaults to 10
)

## Model prediction

In [ ]:
# Make predictions
predictions = model.predict(padded_sequences)

# Get the confidence scores and the predicted classes
confidence_scores = np.max(predictions, axis=-1)  # Maximum probability for each timestep
predicted_classes = np.argmax(predictions, axis=-1)  # Class with the highest probability for each timestep

# Print results
print("Confidence scores shape:", confidence_scores.shape)  # Same shape as the input sequence
print("Predicted classes shape:", predicted_classes.shape)
print(predicted_classes)